In [8]:
import pandas as pd
from gensim.corpora.dictionary import Dictionary
import json
from gensim.utils import tokenize
from gensim.parsing.porter import PorterStemmer
from gensim import models, similarities
import numpy as np
import nltk
import pickle
from gensim.models.coherencemodel import CoherenceModel


In [2]:
# parse a random 12.5% of the data to use 

f = open("amazon/Home_and_Kitchen.json", "r")
f2 = open("amazon/Home_and_Kitchen_8.json", "w")

num_lines_before = 0
num_lines_after = 0
for line in f:
    num_lines_before += 1
    # randomly select the data using hashing
    if hash(line) % 8 == 0:
        f2.write(line)
        num_lines_after += 1

f.close()
f2.close()

print(num_lines_before)
print(num_lines_after)

21928568
2740119


In [3]:
# print out an entry of the meta data to see what I might want to use in the future

f = open("amazon/meta_Home_and_Kitchen.json", "r")

print(f.readline())

f.close()

{"category": ["Home & Kitchen", "Kitchen & Dining", "Dining & Entertaining", "Dinnerware", "Plates", "Dinner Plates"], "tech1": "", "description": ["It was a time honored tradition among the early American families that when someone deserved special praise or attention they were served dinner on the Red Plate. Observe the special occasions of each"], "fit": "", "title": "You Are Special Today Red Plate [With Red Pen]", "also_buy": ["B0001XR2F2", "B01LY51HUN", "B07CXZ9C5B", "0310258952"], "image": [], "tech2": "", "brand": "Waechtersbach USA", "feature": [], "rank": [">#39,665 in Kitchen & Dining (See Top 100 in Kitchen & Dining)", ">#93 in Kitchen & Dining > Tabletop > Dinnerware > Plates > Dinner Plates", ">#19,436 in Home & Kitchen > Kitchen & Dining > Food Service Equipment & Supplies"], "also_view": ["B0001XR2F2", "B00MOFKX1A", "B07G3LN13B", "B07CYXMFFH", "B01LY51HUN", "B06XCK743L", "B00PV4FHZI", "B07C1WTCL4", "B001ARPMD2", "B00847B0IC", "B000NCW2RE", "B06ZZ1GLQR"], "main_cat": "Am

In [4]:

f = open("amazon/Home_and_Kitchen_8.json", "r")
f2 = open("amazon/Home_and_Kitchen_8_new.txt", "w")

# drop all lines with reviews less than 5 characters and only write out the reviewText to a new file 
num_lines = 0

for line in f:
    tmp = json.loads(line)
    try:
        if len(tmp['reviewText']) > 5:
            f2.write(tmp['reviewText'] + "\n")
            num_lines += 1
    except:
        continue

f.close()
f2.close()

print(num_lines)



2703639


In [5]:


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# create stopword dictionary
stops = set(stopwords.words('english'))

# tokenize the reviews from the file created above
with open("amazon/Home_and_Kitchen_8_new.txt", "r") as f:
    tokenized_texts = [list(tokenize(text)) for text in f]
    
print(tokenized_texts[0][:10])
print(len(tokenized_texts))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Erik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['Great', 'product', 'love', 'it']
3588963


In [6]:
# filter the tokenized reviews by dropping all words that are stopwords
filtered = [[w for w in text if not w.lower() in stops] for text in tokenized_texts]

print(filtered[0][:10])

['Great', 'product', 'love']


In [7]:

# try stemming the filtered words 
stemmer = PorterStemmer()
stemmed_texts = [[stemmer.stem(word) for word in text] for text in filtered]

stemmed_texts[0][:10]

['great', 'product', 'love']

In [8]:


# create dictionary based on stemmed words
dictionary = Dictionary(stemmed_texts)
# create corpus from the words in dictionary
corpus = [dictionary.doc2bow(text) for text in stemmed_texts]



In [9]:
# print out the words and their counts for the first review
top_words_in_doc_0 = sorted(corpus[0], key=lambda e: e[1], reverse=True)[:10]
for word_index, count in top_words_in_doc_0:
    print(f'{dictionary[word_index]}   \tindex: {word_index}\tcount: {count:,}')

great   	index: 0	count: 1
love   	index: 1	count: 1
product   	index: 2	count: 1


In [10]:
#check length of our lists
print(len(stemmed_texts))
print(len(dictionary))
print(len(corpus))


3588963
138627
3588963


In [11]:

# create first "base" model with 10 topics. 
# I am first attempting to predict categories of reviews, i.e. distinguish cookware from bedding etc
model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=10, workers=14)

In [12]:
# print out the words associated with each topic and their corresponding values
print(model.show_topics(num_topics=10, num_words=10))

[(0, '0.084*"great" + 0.060*"product" + 0.038*"qualiti" + 0.037*"work" + 0.036*"good" + 0.033*"price" + 0.016*"well" + 0.014*"excel" + 0.014*"expect" + 0.014*"thank"'), (1, '0.030*"coffe" + 0.023*"cup" + 0.020*"us" + 0.014*"water" + 0.014*"make" + 0.013*"mug" + 0.013*"hot" + 0.011*"tea" + 0.011*"keep" + 0.011*"drink"'), (2, '0.026*"us" + 0.018*"sheet" + 0.017*"wash" + 0.015*"well" + 0.014*"pan" + 0.013*"cook" + 0.012*"like" + 0.009*"set" + 0.009*"on" + 0.008*"make"'), (3, '0.019*"on" + 0.016*"mattress" + 0.012*"like" + 0.011*"time" + 0.010*"purchas" + 0.010*"us" + 0.010*"review" + 0.010*"year" + 0.009*"bui" + 0.009*"bought"'), (4, '0.023*"us" + 0.017*"easi" + 0.016*"cut" + 0.014*"chair" + 0.011*"knife" + 0.011*"great" + 0.010*"would" + 0.009*"recommend" + 0.008*"good" + 0.008*"imag"'), (5, '0.027*"us" + 0.017*"water" + 0.017*"clean" + 0.013*"vacuum" + 0.011*"work" + 0.010*"rug" + 0.010*"on" + 0.010*"get" + 0.010*"like" + 0.009*"time"'), (6, '0.043*"love" + 0.020*"perfect" + 0.020*"us" 

In [13]:

# print out the first ten words of each topic learned
for ix in range(10):
    top10 = np.argsort(model.get_topics()[ix])[-10:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

0:  thank expect excel well price good work qualiti product great
1:  drink keep tea hot mug make water us cup coffe
2:  make on set like cook pan well wash sheet us
3:  bought bui year review us purchas time like mattress on
4:  imag good recommend would great knife chair cut easi us
5:  time like get on rug work vacuum clean water us
6:  easi small on bag great fit size us perfect love
7:  great qualiti pictur gift like beauti nice color love look
8:  time piec replac get us work back return would on
9:  smell look light us comfort like great room bed pillow


In [14]:
# trying without stemmed words

# create new corpus based on filtered words instead of stemmed words
dictionary = Dictionary(filtered)

new_corpus = [dictionary.doc2bow(text) for text in filtered]


# retrain the same model with this new corpus and dictionary
model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=10, workers=14)

In [15]:
# again print out top 10 words per topic
for ix in range(10):
    top10 = np.argsort(model.get_topics()[ix])[-10:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

0:  two box used would came get use mattress first time
1:  used good price job clean well easy works use great
2:  like use get mug cold lid hot cup coffee water
3:  bag really love great together like sturdy put easy nice
4:  gift purchase good review received happy would recommend love product
5:  use fit would big like perfect make little small size
6:  unit vacuum even return could buy get back like would
7:  rug daughter great set love beautiful color bed perfect room
8:  cute nice gift look like expected good made price well
9:  oven sheet blanket stick good use like pan soft pillow


In [16]:


# generate the coherence of the topics using both u_mass and c_v
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)


umass:  -3.014806708032802
c_v:  0.5428692998435725


In [17]:
# Now that we have some sort of a baseline, try with less topics

# retrain with half the amount of topics
model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=5, workers=14)

In [18]:
# print the top 10 words for each topic
for ix in range(5):
    top10 = np.argsort(model.get_topics()[ix])[-10:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

0:  like pan great received price used loved gift use product
1:  color would price well love size good product nice great
2:  mattress really together fit bed use like put easy well
3:  light hot back really get time expected would water like
4:  well would make great get cup time use coffee love


In [19]:
# generate corresponding u_mass and c_v for model above

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)

# looks like a possible slight improvement, lets try with more stopwords

umass:  -2.8200038535524636
c_v:  0.517487878292162


In [20]:
# adding more stop words that are not associated with the categories

stops.add("like")
stops.add("love")
stops.add("loved")
stops.add("loves")
stops.add("use")
stops.add("get")
stops.add("used")
stops.add("would")
stops.add("even")

print(stops)


{'yourselves', 'than', 't', 'above', 'shouldn', "mightn't", "don't", 'yourself', 'these', "didn't", "you'll", 'do', 'were', 'wouldn', 'y', 'over', 'be', 'any', 'to', 'where', 'more', 'each', 'your', 'why', 'use', 'haven', 'those', 'out', 'yours', 'or', 'as', 'such', 'about', 'a', 'can', 'needn', 'ma', 'o', 'until', 'own', 'further', "wouldn't", 'get', 'aren', 're', 'this', "you'd", 'having', 'she', 'the', "shan't", "mustn't", 'by', 'now', "won't", 'themselves', 'off', 'mightn', 'their', 'itself', 'too', 'theirs', 'into', 'its', 'ourselves', 'hers', 'my', 'am', 'just', "should've", 'at', "couldn't", 'her', 'shan', 'doesn', 'and', 'himself', 'what', 'not', "wasn't", 'loved', 'ain', 'our', 'him', 'm', 'used', 'will', 'whom', 'before', 'so', 'that', 'which', 'for', "that'll", 'didn', 'was', 'if', 'most', 'has', 'being', 'after', 'in', 'on', 'here', "shouldn't", 'other', 'doing', 'with', 'i', "hadn't", 'he', 'there', 'don', 'did', 'like', 'myself', 'loves', "you've", 'then', 'hasn', 'does',

In [21]:
# generate new dictionary and corpus with the new filtered stopwords that are not stemmed
filtered = [[w for w in text if not w.lower() in stops] for text in tokenized_texts]

print(filtered[0][:10])

dictionary = Dictionary(filtered)
corpus = [dictionary.doc2bow(text) for text in filtered]

['Great', 'product']


In [22]:

# train a new model, again with 5 topics
model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=5, workers=20)

In [23]:
# write out top ten words per topic
for ix in range(5):
    top10 = np.argsort(model.get_topics()[ix])[-10:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')
    
# this doesn't seem too much better than before as I am not seeing super well defined categories    

0:  hot cup pan water Works clean one easy great coffee
1:  really one made easy great perfect Perfect size well small
2:  bed good really little put room one nice great well
3:  buy gift well Great good price one product great quality
4:  work months back unit top first lid water time one


In [24]:
# check the coherence of the above model
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)

# seems like this might be performing just slightly worse

umass:  -2.9400663032110743
c_v:  0.4880056985920172


In [25]:
# Lets see what trying with more topics does

model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=15, workers=14)

In [26]:
# print top ten words per topic
for ix in range(15):
    top10 = np.argsort(model.get_topics()[ix])[-10:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')
    
# can sort of see topics, but still lots of noise

0:  side put soft top fit little well nice one mattress
1:  sheets well good much price bought vacuum one pillow bed
2:  ordered return together put received box color back product one
3:  curtain one price shower small quality good perfect size Great
4:  old first make time ice bought ever best years one
5:  great easy make grinder one maker cups machine cup coffee
6:  warm pillows nice little air Perfect fan chair light room
7:  need easy nice kitchen much table space well great set
8:  product much buy blender Awesome fabric money well great one
9:  top cup mug one glass cold coffee lid hot water
10:  long battery still last first work well unit one time
11:  ref ie product class hook normal B link wine glasses
12:  much one oven good well Easy pan great clean easy
13:  buy Bought rice water year Christmas bought great one gift
14:  looks made nice Great price good well quality product great


In [27]:
# evaluate coherence for above model
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)

umass:  -3.0560961233570483
c_v:  0.5594208013130275


In [28]:
# try with 7 topics

model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=7, workers=14)

In [29]:
# print top ten words per topic
for ix in range(7):
    top10 = np.argsort(model.get_topics()[ix])[-10:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

0:  sheets really room color pillow bed well great soft nice
1:  make time chair bought years made set well one mattress
2:  perfect well vacuum easy bag cute one Great gift great
3:  arrived looks item Great one great price good quality product
4:  unit heat ice back top hot time lid smell water
5:  hot years clock mug great bought time cup coffee one
6:  nice fit perfect small well little one put easy great


In [30]:
# evaluate coherence for above model
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)

umass:  -2.8411466628708757
c_v:  0.5220114469112925


In [31]:
# evaluate on six topics
model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=6, workers=20)

for ix in range(6):
    top10 = np.argsort(model.get_topics()[ix])[-10:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)    
    

0:  good easy pan first great review clean time product one
1:  pretty good pot set product much nice made well one
2:  easy nice works good price well product quality Great great
3:  bag great fit good top well size little small easy
4:  cold great ice make time hot cup one water coffee
5:  pillow back really color nice soft room bed bought one
umass:  -2.8641187015656944
c_v:  0.5053322605290452


In [32]:
# evaluate on four topics
model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=4, workers=20)

for ix in range(4):
    top10 = np.argsort(model.get_topics()[ix])[-10:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)  

0:  room quality time well price bed good product one great
1:  top look looks really little great put well one nice
2:  make much time hot cup one great well water coffee
3:  made gift well good easy product quality great one Great
umass:  -2.7389961461909933
c_v:  0.43561696978755293


In [33]:
# Now that we have tried to do categories with random 10% of the data, lets try with poor ratings only
# and see if we see different topics in the poor ratings


f = open("amazon/Home_and_Kitchen.json", "r")
f2 = open("amazon/Home_and_Kitchen_poor.txt", "w")

num_lines_before = 0
num_lines_after = 0

# iterate through each line and select first 2 million entries that are poor ratings
for line in f:
    if num_lines_after >= 2_500_000:
        break
    num_lines_before += 1
    tmp = json.loads(line)
    try:
        if len(tmp['reviewText']) > 5 and tmp['overall'] == 0.0 or tmp['overall'] == 1.0 or tmp['overall'] == 2.0:
            f2.write(tmp['reviewText'] + "\n")
            num_lines_after += 1
    except:
        continue

f.close()
f2.close()

print(num_lines_before)
print(num_lines_after)

18125643
2500000


In [1]:
# grab original stopwords again

nltk.download('stopwords')
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

print(stops)

{'which', "wasn't", 'ours', 'his', "it's", 'wasn', 'her', 'are', 'from', 'why', 'this', 'did', 'couldn', 'after', 'aren', 'when', 'own', 'isn', 'through', 'yours', 'between', 've', "you'll", 'ain', "shan't", "she's", "you've", 'for', "haven't", 'because', 'again', "mightn't", 'each', 'won', 'ourselves', 'haven', 'their', 'nor', 's', 'whom', 'themselves', 'that', 'should', 'she', 'being', 'itself', 'until', 'shouldn', "mustn't", 'while', 'myself', 'yourselves', 'who', 'into', 'how', 'we', 'be', 'those', 'no', 'then', 'had', 'were', "wouldn't", 'these', 'above', "don't", 'didn', 'up', 'mustn', 'at', 'wouldn', 'herself', 'does', 'needn', 'down', 'o', 'is', 'our', 'm', 'only', 'so', 'few', 'and', 'what', 'he', 'its', 'having', 'shan', 'am', 'doing', 'about', 'there', 'or', 'hers', 'as', 'now', "shouldn't", "hadn't", 'in', 'than', 'here', "isn't", 'has', 'a', 'yourself', 'too', 'y', 'before', 'it', 'me', 'been', 'on', 'same', 'you', 'but', 'an', 'out', 'where', "needn't", 'your', "didn't", 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Erik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

# tokenize the words that are in our poor ratings
with open("amazon/Home_and_Kitchen_poor.txt", "r") as f:
    tokenized_texts = [list(tokenize(text)) for text in f]
    
print(tokenized_texts[0][:10])
print(len(tokenized_texts))

['I', 'truly', 'wanted', 'to', 'like', 'this', 'item', 'It', 'fell', 'apart']
3673242


In [5]:
# filter the tokenized words based on our stopwords
filtered = [[w for w in text if not w.lower() in stops] for text in tokenized_texts]

dictionary = Dictionary(filtered)


print(filtered[0][:10])

['truly', 'wanted', 'like', 'item', 'fell', 'apart', 'installing', 'screwdriver']


In [6]:
# create a new corpus based on these words
new_corpus = [dictionary.doc2bow(text) for text in filtered]

# train an lda model with our new corpus and try to find five topics. This model I am looking to see if there
# are topics within the lower ratings of what issues there were with the products
model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=5, workers=20)

In [9]:
# print top 8 words per topic
for ix in range(5):
    top8 = np.argsort(model.get_topics()[ix])[-8:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

# calculate corresponding coherence
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)  

# looks like this does decently well compared to the original models

0:  work well get like one vacuum would use
1:  get buy good use would quality product one
2:  product work working would money months time one
3:  return made back quality product would one like
4:  time lid get one like coffee use water
umass:  -2.5768663320964125
c_v:  0.5519508221535441


In [10]:
# try adding some common words from previous model that don't have to do with the type of negative review
stops.add('one')
stops.add('would')

# create new filtered list and dictionary with these new words
filtered = [[w for w in text if not w.lower() in stops] for text in tokenized_texts]

dictionary = Dictionary(filtered)

print(filtered[0][:10])

['truly', 'wanted', 'like', 'item', 'fell', 'apart', 'installing', 'screwdriver']


In [11]:
# train new model with a new corpus incorporating those two words
# still using five topics
new_corpus = [dictionary.doc2bow(text) for text in filtered]

model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=5, workers=20)

In [12]:
# print top eight words from each topic
for ix in range(5):
    top8 = np.argsort(model.get_topics()[ix])[-8:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

# calculate coherence for above model
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence) 

# words are maybe more related to types of issues than previous model? Not sure

0:  even mattress like pillow work get time use
1:  get use time water working product months coffee
2:  put together broken came box back return product
3:  picture small money made cheap product quality like
4:  smell first time like pan used water use
umass:  -2.677733855565883
c_v:  0.5329744738285094


In [13]:
# try decreasing the number of topics and retrain
model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=4, workers=20)

for ix in range(4):
    top8 = np.argsort(model.get_topics()[ix])[-8:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence) 

# hard to tell if words are more related to different types of issues
# metrics seem similar but c_v seems to be a little worse than before

0:  cup hot back use get like water coffee
1:  work picture color size product get fit like
2:  working get months time work product money use
3:  cheap back like use time made product quality
umass:  -2.617979763107618
c_v:  0.4593826466756179


In [14]:
# maybe try increasing the number of topics?
model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=6, workers=20)

for ix in range(6):
    top8 = np.argsort(model.get_topics()[ix])[-8:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence) 

# I think these words are slightly more related within the topic than before and the 
# metrics improved slighly in the c_v calculation



0:  pan get like cup time water use coffee
1:  arrived broken like together put box back product
2:  fan use months used unit product work water
3:  good bed cheap mattress pillow product quality like
4:  money broke buy plastic like made use get
5:  use get back product money return time months
umass:  -2.5679934661349697
c_v:  0.5436647060477481


In [18]:
# print out topic distributions for first five documents
for i in range(5):
    print(filtered[i])
    print(model.get_document_topics(new_corpus[i]))
    print()
    

['truly', 'wanted', 'like', 'item', 'fell', 'apart', 'installing', 'screwdriver']
[(0, 0.01863869), (1, 0.9066615), (2, 0.018665267), (3, 0.0187239), (4, 0.018683974), (5, 0.01862667)]

['Instructions', 'wrong', 'sturdy']
[(0, 0.041810527), (1, 0.79051197), (2, 0.04180895), (3, 0.041887335), (4, 0.04206354), (5, 0.04191765)]

['like', 'coffee', 'hot', 'borderline', 'scorching', 'drinkable', 'years', 'ago', 'purchased', 'several', 'Avantro', 'coffee', 'mugs', 'similar', 'able', 'keep', 'coffee', 'preferred', 'drinking', 'temperature', 'well', 'hours', 'even', 'coldest', 'days', 'work', 'outside', 'hot', 'coffee', 'crucial', 'nasty', 'New', 'England', 'winters']
[(0, 0.6076076), (5, 0.3727433)]

['Well', 'day', 'new', 'mug', 'quite', 'disappointed', 'mug', 'barely', 'kept', 'coffee', 'warm', 'hour', 'quite', 'unacceptable', 'seeing', 'company', 'bought', 'Avantro']
[(0, 0.5107896), (5, 0.45276675)]

['mug', 'fair', 'job', 'keeping', 'coffee', 'hot', 'absolutely', 'protect', 'shirt', 'pan

In [19]:
# decided that this was the most coherent of the models and pickled it

with open("topic.model", "wb") as f:
    pickle.dump(model, f)

In [46]:
# see if increasing the topics again keeps improving
model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=7, workers=20)

for ix in range(7):
    top8 = np.argsort(model.get_topics()[ix])[-8:]
    print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence) 

# still seems pretty coherent, but previous model might be slightly better

0:  clean like air much small use vacuum mattress
1:  make get like time used product pan use
2:  box received item disappointed broken use quality product
3:  fit quality time even put product get like
4:  new get back product Amazon months working return
5:  get use money like cheap made broke plastic
6:  hot like use back work time water coffee
umass:  -2.627010918628895
c_v:  0.5202661200062719
